## Initial steps
1. File -> New -> Terminal
2. In the terminal, type following commands
        mkdir med264/
        cd med264/
        wget https://archive.physionet.org/users/shared/challenge-2019/training_setB.zip
        unzip training_setB.zip

## Install & Load Dependencies

In [1]:
dependencies  <- c("tidyverse", "tidymodels")

for (package in dependencies) {
    if (!(package %in% installed.packages()['Package'])) {
        install.packages(package)
    }
    
    library(package, character.only = TRUE)
}



Installing package into ‘/home/aaron/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.1     ✔ dplyr   1.0.5
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Installing package into ‘/home/aaron/R/x86_64-pc-linux-gnu-library/3.6’
(as ‘lib’ is unspecified)

── Attaching packages ────────────────────────────────────── tidymodels 0.1.3 ──

✔ broom        0.7.6      ✔ rsample      0.0.9 
✔ dials        0.0.9      ✔ tune         0.1.5 
✔ infer        0.5.4      ✔ workflows    0.2.2 
✔ modeldata    0.1.0      ✔ workflowsets 0.0.2 
✔ parsnip      0.1.5      ✔ yardstick    0.0.8 
✔ recipes      0.1.16     

── Conflicts ───────────────────────────────────────── tidymodels_

## Data processing
1. Load the list of files in the input directory
2. Split the records to training and testing set
3. Convert the data into a numpy array

In [2]:
# Name of the input directory
input_directory = 'training_setB'

# Read all PSV files in the input directory and convert to a tibble
all_data  <- list.files(path = input_directory, full.names = TRUE) %>% 
    .[grepl(".psv", .)] %>%
    map(., read_delim, delim = "|", col_types = cols()) %>% 
    bind_rows() %>% 
    mutate(ID = row_number())

# Split the data into training and testing set
# 80% of data -> Training
# 20% of data -> Testing

train_all  <- slice_sample(all_data, prop = 0.8)

test_all  <- all_data %>%
    anti_join(train_all, by="ID")

train_labels  <- train_all["SepsisLabel"]
test_labels  <- test_all["SepsisLabel"]

train_data <- select(train_all, -c("SepsisLabel", "ID"))
test_data <- select(test_all, -c("SepsisLabel", "ID"))

## Standardization of data
1. Compute the mean and standard deviation of ONLY training set
2. Use the statistics computed to standardize Training and Testing sets

In [3]:
x_mean  <- colMeans(train_data, na.rm = TRUE)
x_sd  <- apply(train_data, 2, sd, na.rm = TRUE)

# For NaN entries, replace with 0
# For the remaining entries, standardize with mean and std
train_data <- sweep(train_data, 2, x_mean, "-") %>%
    sweep(., 2, x_sd, "/")  %>%
    `[<-`(., is.na(.), value = 0)

test_data <- sweep(test_data, 2, x_mean, "-") %>%
    sweep(., 2, x_sd, "/") %>% 
    `[<-`(., is.na(.), value = 0)
    

## Model training
1. We will use a simple logistic regression model to model the data

### Exercise 1
1. Use a different penalty for Logistic Regression (LR) and plot the AUC curves
2. Use a Support Vector Machine classifier instead of LR and plot the AUC curves
3. Use a Random Forest Classifier instead of LR and plot the AUC curves 

        https://cran.r-project.org/web/packages/caret/caret.pdf

In [5]:
colMeans(test_data)

HR            O2Sat             Temp              SBP 
   -4.273077e-04    -5.055635e-04    -5.242326e-04     4.295242e-04 
             MAP              DBP             Resp            EtCO2 
    1.560220e-03     2.724224e-03    -1.691664e-03    -6.439992e-04 
      BaseExcess             HCO3             FiO2               pH 
   -1.847088e-04    -1.463365e-04    -1.997020e-04     1.134440e-03 
           PaCO2             SaO2              AST              BUN 
   -5.078340e-04     2.193932e-04    -2.442690e-04    -1.475458e-04 
    Alkalinephos          Calcium         Chloride       Creatinine 
    3.188209e-04    -9.564654e-04     4.806766e-06     4.011072e-04 
Bilirubin_direct          Glucose          Lactate        Magnesium 
   -2.904529e-05    -5.684313e-04     1.144708e-04    -2.528438e-05 
       Phosphate        Potassium  Bilirubin_total        TroponinI 
   -2.513062e-04    -2.405567e-04     5.589332e-04     3.596745e-04 
             Hct              Hgb              PTT              WBC 
   -1.647102e-04    -1.776498e-04    -3.460507e-04     3.073605e-04 
      Fibrinogen        Platelets              Age           Gender 
    3.399097e-04    -1.593247e-03     9.164044e-04    -1.296870e-03 
           Unit1            Unit2      HospAdmTime           ICULOS 
   -6.346549e-04     6.346549e-04    -2.860924e-04    -2.274126e-03